In [12]:
import cv2
import numpy as np
from math import pi
from scipy.ndimage import filters
from numpy import floor

In [2]:
img = cv2.imread('lena30.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
def ratio(x):
    # x[4] - центральный элемент
    value = 0.0
    if x[4] == 0:
        value = pi / 2.0
    else:
        value = np.arctan(float(x.sum() - 9.0 * x[4]) / x[4])
    return value

def intense_fraction(x):
    value = 0.0
    if x[5] == x[1]:
        value = 2.0 * pi
    else:
        value = 2.0 * np.arctan(float(x[7] - x[3]) / (x[5] - x[1])) + pi
    return value

In [33]:
# Magic numbers
M = 4
T = 8
S = 4
eps = 0.000001
block_size = 12

In [48]:
faces = face_cascade.detectMultiScale(gray_img)
for (x, y, w, h) in faces:
    # here starts new block
    image = gray_img[y:y + h, x:x + w]
   
    # Calculating differential excitation
    image_de = filters.generic_filter(image, ratio, (3, 3))
   
    # Calculating gradient orientation
    image_teta = filters.generic_filter(image, intense_fraction, (3, 3))

    # Calculating dominant orientation
    phi = 2.0 * pi * (floor((image_teta * T) / (2.0 * pi) + 1.0 / 2.0) % T) / T
    # Each phi[j] is from T values: {phi_0, phi_1, ....phi_(T-1)}
    H_t = []
    for t in range(T):
        phi_t = 2.0 * pi * t / T 
        # Indicies x and y coordinates   
        indicies = np.where(abs(phi - phi_t) < eps)
        # We wil build hostogram on diff excitiation
        values = image_de[indicies[0], indicies[1]]
        # In H_t (array_of_values, array_of_coords)
        H_t.append(np.histogram(values, bins=M * S))
    
    # Divide each of t histograms to m parts
    length = len(H_t[0][0])
    print "length = " + str(length)
    print "first hist:"
    print H_t[0]
    # -----> t
    # |
    # |
    # |
    # |
    # v m
    H_tm = []
    for t in range(T):
        hist_part = []
        for m in range(M):
            hist_part.append((H_t[t][0][m * length / M: (m + 1) * length / M],
                             H_t[t][1][m * length / M: (m + 1) * length / M]))
        H_tm.append(hist_part)

    H_m = []
    for m in range(M):
        hist_m = []
        for t in range(T):
            # NOTE: taking only values
            hist_m += [value for value in H_tm[t][m][0]]
        H_m.append(hist_m)

    H = []
    for i in range(M):
        H += H_m[i]
        
    print H

length = 16
first hist:
(array([4607,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,   21], dtype=int64), array([   0.    ,   15.9375,   31.875 ,   47.8125,   63.75  ,   79.6875,
         95.625 ,  111.5625,  127.5   ,  143.4375,  159.375 ,  175.3125,
        191.25  ,  207.1875,  223.125 ,  239.0625,  255.    ]))
[4607, 0, 0, 0, 1929, 0, 0, 0, 0, 0, 0, 0, 1759, 0, 0, 0, 4674, 0, 0, 0, 9920, 0, 0, 0, 5570, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 10, 0, 0, 0, 7, 0, 0, 0, 11, 0, 0, 0, 0]
